In [1]:
import sys, os
print("Kernel Python:", sys.executable)

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
print("PyTorch:", torch.__version__)

from tqdm import tqdm
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print("Current location:", os.getcwd())

from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig

import torch
print(torch.version.cuda)   # shows the CUDA version PyTorch was built against
print(torch.backends.cudnn.version())  # cuDNN version
print(torch.cuda.is_available())

Kernel Python: /zhome/c9/0/203261/miniconda3/envs/esm_gpu/bin/python
PyTorch: 2.5.1
Using device: cuda
Current location: /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts
12.1
90100
True


In [15]:
import pandas as pd

interaction_df = pd.read_csv("../data/PPint_DB/disordered_interfaces_no_cutoff_filtered_nonredundant80_3å_5.csv.gz",index_col=0).reset_index(drop=True)
sample_df = interaction_df.sample(n=500, random_state=0)
all_pdbs = set(sample_df['PDB'])
interactiondf_filtered = interaction_df[interaction_df['PDB'].isin(all_pdbs)].copy()
interactiondf_filtered

,PDB,interface_index,chainname,sequence,interface_residues,disorderscore_predictions,mean disorder interface,seq. len,PDB_interface_name,protien_interface_sequences,interacting_seq_aa,inter_chain_hamming
162,7OOP,3,D,EEDASQLIFPKEFETAETLLNSEVHMLLEHRKQQNESAEDEQELSE...,"[1, 2, 3, 4, 15, 16, 17, 18, 19, 20, 23, 35, 4...","[0.4348, 0.4415, 0.4386, 0.4392, 0.4268, 0.425...",0.181523,128,7OOP_3,EEDASQLIFPKEFETAETLLNSEVHMLLEHRKQQNESAEDEQELSE...,EDASAETLLNVEQSVFKTLNYTEEALIDQT-FYHITGIFKGEVIGPDY,0.187135
163,7OOP,3,G,MFYHISLEHEILLHPRYFGPNLLNTVKQKLFTEVEGTCTGKYGFVI...,"[1, 2, 3, 4, 31, 35, 45, 79, 80, 81, 82, 83, 1...","[0.187, 0.1449, 0.1199, 0.13, 0.1238, 0.1177, ...",0.053141,171,7OOP_3,EEDASQLIFPKEFETAETLLNSEVHMLLEHRKQQNESAEDEQELSE...,EDASAETLLNVEQSVFKTLNYTEEALIDQT-FYHITGIFKGEVIGPDY,0.187135
164,7OOP,10,U,GNDLYFVKLPNFLSVEPRPFDPQYYEDEFEEEGRTRLKLKVENNAR...,"[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 15, 16, 17...","[0.3113, 0.3175, 0.3417, 0.3262, 0.3251, 0.321...",0.132819,104,7OOP_10,ERSGVVCRVKYCNSLPDIPFDPKFITYPFDQNRFVQYKATSLEKQH...,KDRQAIKTFDAYVTPVEVMPVSQAMIRGMDQEADFFIFREGDYNEL...,0.202765
165,7OOP,10,V,ERSGVVCRVKYCNSLPDIPFDPKFITYPFDQNRFVQYKATSLEKQH...,"[89, 90, 91, 94, 97, 98, 100, 101, 102, 104, 1...","[0.3385, 0.322, 0.3194, 0.3279, 0.3269, 0.3176...",0.154636,217,7OOP_10,ERSGVVCRVKYCNSLPDIPFDPKFITYPFDQNRFVQYKATSLEKQH...,KDRQAIKTFDAYVTPVEVMPVSQAMIRGMDQEADFFIFREGDYNEL...,0.202765
166,7OOP,11,V,ERSGVVCRVKYCNSLPDIPFDPKFITYPFDQNRFVQYKATSLEKQH...,"[17, 18, 19, 20, 22, 23, 24, 25]","[0.3385, 0.322, 0.3194, 0.3279, 0.3269, 0.3176...",0.082950,217,7OOP_11,ERSGVVCRVKYCNSLPDIPFDPKFITYPFDQNRFVQYKATSLEKQH...,IPFDKFIT-RERVRTR,0.133641
...,...,...,...,...,...,...,...,...,...,...,...,...
49251,7ARC,11,r,PKLVYDIKYFARDYRRNNSYTVRSVDSKTPFDYSKVFGSAPLKPAD...,"[10, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2...","[0.3286, 0.336, 0.3164, 0.2952, 0.2916, 0.289,...",0.245427,60,7ARC_11,FKAVADVEIDFSNRAVLKKYVGVDHLSKEQGTKAKFIVSLNSLLEA...,EAYKVENAEVAYSAPVFDYVDASL-AYNNSYTVRSVDSKPYFPLKPV,0.208955
49430,6WDS,0,L,SVLTQPPSASGTPGQRVTISCSGSSSNIEYNYVYWYQKFPGTAPKL...,"[33, 41, 42, 43, 44, 45, 48, 88, 89, 90, 92, 9...","[0.7922, 0.8499, 0.8625, 0.855, 0.8491, 0.8191...",0.116711,106,6WDS_0,SVLTQPPSASGTPGQRVTISCSGSSSNIEYNYVYWYQKFPGTAPKL...,YTAPKLYAAWDISGVVFG-GLEWFYAYRAYNLWGQG,0.217391
49431,6WDS,0,H,VQLVESGGGLVKPGGLRLSCAASGFTFSTYIMTWVRQAPGRGLEWV...,"[41, 42, 43, 44, 56, 57, 58, 92, 100, 101, 102...","[0.7256, 0.7606, 0.7419, 0.7242, 0.7088, 0.671...",0.074329,115,6WDS_0,SVLTQPPSASGTPGQRVTISCSGSSSNIEYNYVYWYQKFPGTAPKL...,YTAPKLYAAWDISGVVFG-GLEWFYAYRAYNLWGQG,0.217391
49432,6WDS,2,B,SDRVLQLKLGNSAIVTQEAANYCCAYGEWPNYLPDHEAVAIDKPTQ...,"[20, 21, 22, 23, 24, 25, 26, 27, 36, 63, 64, 6...","[0.2451, 0.2123, 0.2245, 0.2213, 0.2161, 0.218...",0.106527,238,6WDS_2,GVPTYLLPGSGQFLTTDDHSSAPALPCFNPTPEMHIPGQVRNMLEV...,MIPGVESMMGMLKNTLCGSFMLSSESSSH-NYCCAYGEETGSTKFH...,0.155102


In [16]:
interaction_Dict = {}

for _, row in interactiondf_filtered.iterrows():
    key = row['PDB']   
    seq = row['sequence']                    
    interaction_Dict.setdefault(key, []).append(seq)

list(interaction_Dict.items())[:2]

[('7OOP',
  ['EEDASQLIFPKEFETAETLLNSEVHMLLEHRKQQNESAEDEQELSEVFMKTLNYTARFSRFKNRETIASVRSLLLQKKLHKFELACLANLCPETAEESKALIPSLEGRFEDEELQQILDDIQTKRSFQ',
   'MFYHISLEHEILLHPRYFGPNLLNTVKQKLFTEVEGTCTGKYGFVIAVTTIDNIGAGVIQPGRGFVLYPVKYKAIVFRPFKGEVVDAVVTQVNKVGLFTEIGPMSCFISRHSIPSEMEFDPNSNPPCYKTMDEDIVIQQDDEIRLKIVGTRVDKNDIFAIGSLMDDYLGLV',
   'GNDLYFVKLPNFLSVEPRPFDPQYYEDEFEEEGRTRLKLKVENNARIVKWSDGSMSLHLGNEVFDVYKAPLQGDHNHLFIRQGTGLQGQAVFKTKLTFRPHSTD',
   'ERSGVVCRVKYCNSLPDIPFDPKFITYPFDQNRFVQYKATSLEKQHKHDLLTEPDLGVTIDLINPDTYRIDPNVLLDPADEKLLEEEIQKDRDSQITAIEKTFEDAQKSISQHYSKPRVTPVEVMPVSQAMIRGMMDEEGNQFVAYFLPVEETLKKRKRDQEEEMDYAPDDVYDYKIAREYNWNVKNKASKFFIFREGDGVYYNELETRVRLSKRRA',
   'ERSGVVCRVKYCNSLPDIPFDPKFITYPFDQNRFVQYKATSLEKQHKHDLLTEPDLGVTIDLINPDTYRIDPNVLLDPADEKLLEEEIQKDRDSQITAIEKTFEDAQKSISQHYSKPRVTPVEVMPVSQAMIRGMMDEEGNQFVAYFLPVEETLKKRKRDQEEEMDYAPDDVYDYKIAREYNWNVKNKASKFFIFREGDGVYYNELETRVRLSKRRA',
   'VDVTRDIVSRERVWRTRTTILQSTGKNFSKNIFAILQSVKARE']),
 ('5FHB',
  ['QVQLQESGPGLVKPSDTLSLTCTVSGGSLSSFYWSWIRQPPGKGLEWIGYIY

In [17]:
client = ESMC.from_pretrained("esmc_600m", device=torch.device("cpu")).to("cpu")

# Moving model to cuda
client_gpu = client.to("cuda")

def calculate_ESM_pr_res_embeddings(sequence):
    protein = ESMProtein(sequence=sequence)
    protein_tensor = client_gpu.encode(protein)
    logits_output = client_gpu.logits(
    protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
    )
    return logits_output.embeddings.detach().cpu().numpy()

/zhome/c9/0/203261/miniconda3/envs/esm_gpu/lib/python3.10/site-packages/esm/pretrained.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [18]:
for name, seqs in interaction_Dict.items():
    print(seqs)
    break

['EEDASQLIFPKEFETAETLLNSEVHMLLEHRKQQNESAEDEQELSEVFMKTLNYTARFSRFKNRETIASVRSLLLQKKLHKFELACLANLCPETAEESKALIPSLEGRFEDEELQQILDDIQTKRSFQ', 'MFYHISLEHEILLHPRYFGPNLLNTVKQKLFTEVEGTCTGKYGFVIAVTTIDNIGAGVIQPGRGFVLYPVKYKAIVFRPFKGEVVDAVVTQVNKVGLFTEIGPMSCFISRHSIPSEMEFDPNSNPPCYKTMDEDIVIQQDDEIRLKIVGTRVDKNDIFAIGSLMDDYLGLV', 'GNDLYFVKLPNFLSVEPRPFDPQYYEDEFEEEGRTRLKLKVENNARIVKWSDGSMSLHLGNEVFDVYKAPLQGDHNHLFIRQGTGLQGQAVFKTKLTFRPHSTD', 'ERSGVVCRVKYCNSLPDIPFDPKFITYPFDQNRFVQYKATSLEKQHKHDLLTEPDLGVTIDLINPDTYRIDPNVLLDPADEKLLEEEIQKDRDSQITAIEKTFEDAQKSISQHYSKPRVTPVEVMPVSQAMIRGMMDEEGNQFVAYFLPVEETLKKRKRDQEEEMDYAPDDVYDYKIAREYNWNVKNKASKFFIFREGDGVYYNELETRVRLSKRRA', 'ERSGVVCRVKYCNSLPDIPFDPKFITYPFDQNRFVQYKATSLEKQHKHDLLTEPDLGVTIDLINPDTYRIDPNVLLDPADEKLLEEEIQKDRDSQITAIEKTFEDAQKSISQHYSKPRVTPVEVMPVSQAMIRGMMDEEGNQFVAYFLPVEETLKKRKRDQEEEMDYAPDDVYDYKIAREYNWNVKNKASKFFIFREGDGVYYNELETRVRLSKRRA', 'VDVTRDIVSRERVWRTRTTILQSTGKNFSKNIFAILQSVKARE']


In [19]:
# where to save
path_to_output_embeddings = "/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings"
os.makedirs(path_to_output_embeddings, exist_ok=True)

# helper: convert torch tensor to numpy
def to_numpy(x):
    try:
        return x.detach().cpu().numpy()
    except AttributeError:
        return np.asarray(x)

seq_num = ["A", "B", "C", "D", "E", "F", "G", "H"]
for name, seqs in tqdm(interaction_Dict.items(), total=len(interaction_Dict.items()), desc="Embedding targets"):
     for idx, seq in enumerate(seqs[:8]):
        emb = calculate_ESM_pr_res_embeddings(seq)
        emb_np = to_numpy(emb)
        out_path = os.path.join(path_to_output_embeddings, f"{name}_{seq_num[idx]}.npy")
        np.save(out_path, emb_np)
        print(f"Protein {name} embedded and saved to {out_path}")

# Count files in the 
# path_to_output_embeddings = "/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/meta_analysis/targets_embeddings"
if len(os.listdir(path_to_output_embeddings)) == len(interaction_Dict.keys()):
    print("All sequences encoded!")

Embedding targets:   0%|                                                                        | 0/495 [00:00<?, ?it/s]

Protein 7OOP embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7OOP_A.npy
Protein 7OOP embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7OOP_B.npy
Protein 7OOP embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7OOP_C.npy


Embedding targets:   0%|▏                                                               | 1/495 [00:00<03:58,  2.07it/s]

Protein 7OOP embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7OOP_D.npy
Protein 7OOP embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7OOP_E.npy
Protein 7OOP embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7OOP_F.npy


Embedding targets:   0%|▎                                                               | 2/495 [00:00<02:33,  3.22it/s]

...

Protein 3BB5 embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/3BB5_B.npy
Protein 7ARC embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7ARC_A.npy
Protein 7ARC embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7ARC_B.npy


Embedding targets: 100%|█████████████████████████████████████████████████████████████▊| 494/495 [01:42<00:00,  3.98it/s]

Protein 7ARC embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7ARC_C.npy
Protein 7ARC embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/7ARC_D.npy
Protein 6WDS embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/6WDS_A.npy
Protein 6WDS embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/6WDS_B.npy
Protein 6WDS embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/6WDS_C.npy


Embedding targets: 100%|██████████████████████████████████████████████████████████████| 495/495 [01:43<00:00,  4.79it/s]

Protein 6WDS embedded and saved to /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/PPint_DB/ESM_C_embeddings/6WDS_D.npy
